In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from climakitae.explore.amy import get_climate_profile

In [ ]:
selection= {
    "variable": "Air Temperature at 2m",
    "resolution": "3 km",
    "warming_level": [2.0],
    "units": "degF",
    # cached_area=['Los Angeles County']
    "latitude": (37.704432 - 0.05, 37.704432 + 0.05),
    "longitude": (-121.898696 - 0.05, -121.898696 + 0.05),
    "q": 0.95,
}

In [ ]:
get_climate_profile(**selection)